In [36]:
import pandas as pd
import numpy as np

# paths to both files
StudentFile = 'C:/Users/leala/Documents/Data Git Repo/python-challenge/Python_Challenge/Resources/students_complete.csv'
SchoolFile = 'C:/Users/leala/Documents/Data Git Repo/python-challenge/Python_Challenge/Resources/schools_complete.csv'

# open the files
student_data_raw = pd.read_csv(StudentFile)
school_data_raw = pd.read_csv(SchoolFile)

In [37]:
# Adding columns I'll need before merging.  Feels like the cheaters way out but it works since it only applies to school data
school_data_raw["Spending Per Student"] = school_data_raw["budget"] / school_data_raw["size"]

# I'll use this for the spending per school later on, might as well bin them now
spending_bins = [0, 585, 615, 645, 675]
spending_labels = ["<$585", "$585-615", "$615-645", "$645-675"]
school_data_raw["Spending Classification"] = pd.cut(school_data_raw["Spending Per Student"], spending_bins, labels=spending_labels)

# And this takes care of the bins for the size portion
size_bins = [0, 1501, 3001, 5000]
size_labels = ["Small", "Mid Sized", "Large"]
school_data_raw["Size Classification"] = pd.cut(school_data_raw["size"], size_bins, labels=size_labels)

In [38]:
# Clean up Columns and rename them to remove underscores
school_clean = school_data_raw.rename(columns={"school_name":"School Name", "type":"Type", 
                                               "size":"Size", "budget":"Budget"})
student_clean = student_data_raw.rename(columns={"school_name":"School Name", "student_name":"Student Name", 
                                                 "gender":"Gender", "grade":"Grade", "reading_score":"Reading Score", 
                                                 "math_score":"Math Score"})

# merge the two datasets on school name commonality
complete_data = pd.merge(student_clean, school_clean, how="left", on=["School Name", "School Name"])

# I'll eventually need these bins for pass/fail classification so moving them here to clean up each ask
pass_fail= [0, 69, 101]
pass_fail_labels = ['Failed', 'Passed']

complete_data["Pass/Fail Math"] = pd.cut(complete_data["Math Score"], pass_fail, labels=pass_fail_labels)
complete_data["Pass/Fail Reading"] = pd.cut(complete_data["Reading Score"], pass_fail, labels=pass_fail_labels)

# this column is because I can't get my dataframes to put numbers that don't change inside of it
complete_data["Cheat"] = int(1)

In [39]:
#total number of schools, students, budget
total_schools = len(school_data_raw["school_name"])
total_students = sum(school_data_raw["size"])
total_budget = sum(school_data_raw["budget"])

# find the averages of the math/reading scores and make them variables  -- "%.2f" --
math_average = complete_data['Math Score'].mean()
reading_average = complete_data['Reading Score'].mean()

# % passing section
    # determining % pass/fail based on created columns
math_PF = complete_data["Pass/Fail Math"].value_counts()
passing_math = math_PF[1] / total_students
reading_PF = complete_data["Pass/Fail Reading"].value_counts()
passing_reading = (reading_PF[1] / total_students)

# Overall passing rate calculations
overall_passing = (passing_math + passing_reading) / 2

In [40]:
# create a DataFrame for the summary calculations
district_summary = pd.DataFrame({"Total Schools": total_schools, 
                                "Total Students": '{:,}'.format(total_students),
                                "Total Budget": '${:,}'.format(total_budget), 
                                 "Average Math Score": [math_average], 
                                 "Average Reading Score": [reading_average],
                                "Percent Passing Math": '{:.2%}'.format(passing_math),
                                "Percent Passing Reading": '{:.2%}'.format(passing_reading), 
                                "Overall Passing Rate": '{:.2%}'.format(overall_passing)})
district_summary.round(2)

,Average Math Score,Average Reading Score,Overall Passing Rate,Percent Passing Math,Percent Passing Reading,Total Budget,Total Schools,Total Students
0,78.99,81.88,80.39%,74.98%,85.81%,"$24,649,428",15,"39,170"


In [41]:
# total number of schools, students, budget
sum_df = complete_data.groupby(["School Name", "Type"])
school_students = sum_df.sum()["Size"] / sum_df.sum()["Cheat"]
school_budget = sum_df.sum()["Budget"] / sum_df.sum()["Cheat"]

# find the averages of the math/reading scores and make them variables  -- "%.2f" --
school_math_average = sum_df.mean()['Math Score']
school_reading_average = sum_df.mean()['Reading Score']

# determining % pass/fail based on created columns
school_math_PF = sum_df["Pass/Fail Math"].value_counts()
school_passing_math = school_math_PF[1] / school_students
school_reading_PF = sum_df["Pass/Fail Reading"].value_counts()
school_passing_reading = school_reading_PF[1] / school_students
    # ^^^^ NEED TO FORMAT AS PERCENTAGES!!!

# Overall passing rate calculations
school_overall_passing = (school_passing_math + school_passing_reading) / 2
    # ^^^^ NEED TO FORMAT AS PERCENTAGES!!!

In [42]:
sum_data = complete_data.set_index('School Name',inplace = True, drop=False)
sum_data

In [43]:
schools = pd.DataFrame({"Average Math Score": school_math_average, 
                       "Average Reading Score": school_reading_average, 
                       "Total Students": school_students,
                       "Budget": school_budget,
                       "Percent Passing Math": school_passing_math,
                       "Percent Passing Reading": school_passing_reading,
                       "Overall Passing Rate": school_overall_passing})             
schools.round()

,,Average Math Score,Average Reading Score,Budget,Overall Passing Rate,Percent Passing Math,Percent Passing Reading,Total Students
School Name,Type,,,,,,,
Bailey High School,District,77.0,81.0,3124928.0,0.0,0.0,0.0,4976.0
Cabrera High School,Charter,83.0,84.0,1081356.0,1.0,1.0,0.0,1858.0
Figueroa High School,District,77.0,81.0,1884411.0,0.0,1.0,0.0,2949.0
Ford High School,District,77.0,81.0,1763916.0,0.0,1.0,0.0,2739.0
Griffin High School,Charter,83.0,84.0,917500.0,1.0,1.0,1.0,1468.0
Hernandez High School,District,77.0,81.0,3022020.0,0.0,0.0,0.0,4635.0
Holden High School,Charter,84.0,84.0,248087.0,3.0,4.0,2.0,427.0
Huang High School,District,77.0,81.0,1910635.0,0.0,1.0,0.0,2917.0
Johnson High School,District,77.0,81.0,3094650.0,0.0,0.0,0.0,4761.0


In [44]:
sorted_summary = schools.sort_values("Overall Passing Rate")
sorted_summary.tail()

,,Average Math Score,Average Reading Score,Budget,Overall Passing Rate,Percent Passing Math,Percent Passing Reading,Total Students
School Name,Type,,,,,,,
Shelton High School,Charter,83.359455,83.725724,1056600.0,0.726008,0.941511,0.510505,1761.0
Thomas High School,Charter,83.418349,83.848930,1043130.0,0.781957,1.014067,0.549847,1635.0
Griffin High School,Charter,83.351499,83.816757,917500.0,0.870913,1.129428,0.612398,1468.0
Pena High School,Charter,83.839917,84.044699,585858.0,1.329002,1.723493,0.934511,962.0
Holden High School,Charter,83.803279,83.814988,248087.0,2.994145,3.882904,2.105386,427.0


In [31]:
#  *****MATH SCORES BY GRADE*****
grades_list = complete_data.groupby(['School Name', 'Grade'])

math_summary = pd.DataFrame(grades_list["Math Score"].sum())
math_summary.head()

C:\Users\leala\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: 'School Name' is both a column name and an index level.
Defaulting to column but this will raise an ambiguity error in a future version
  


Math Score
School Name         Grade            
Bailey High School  10th        95399
                    11th        96972
                    12th        78634
                    9th        112388
Cabrera High School 10th        38750

In [45]:
#  *****READING SCORES BY GRADE*****
reading_summary = pd.DataFrame(grades_list["Reading Score"].sum())
reading_summary.head()

Reading Score
School Name         Grade               
Bailey High School  10th          100244
                    11th          101263
                    12th           83178
                    9th           118540
Cabrera High School 10th           39262

In [46]:
spending_group = complete_data.groupby(['Spending Classification'])

spending_students = spending_group.sum()["Size"] / spending_group.sum()["Cheat"]

spending_math_PF = spending_group["Pass/Fail Math"].value_counts()
spending_passing_math = spending_math_PF[1] / spending_students
spending_reading_PF = spending_group["Pass/Fail Reading"].value_counts()
spending_passing_reading = spending_reading_PF[1] / spending_students

# Overall passing rate calculations
spending_overall_passing = (spending_passing_math + spending_passing_reading) / 2

In [47]:
spending_summary = pd.DataFrame({"Reading Average": spending_group["Reading Score"].mean().round(2),
                                "Math Average": spending_group["Reading Score"].mean().round(2),
                                "Percent Passing Math": spending_passing_math,
                                "Percent Passing Reading": spending_passing_reading,
                                "Overall Passing Rate": spending_overall_passing})
spending_summary

,Math Average,Overall Passing Rate,Percent Passing Math,Percent Passing Reading,Reading Average
Spending Classification,,,,,
<$585,83.96,0.161221,0.211273,0.111169,83.96
$585-615,83.84,0.206935,0.271180,0.142691,83.84
$615-645,81.43,0.087997,0.115316,0.060677,81.43
$645-675,81.01,0.071550,0.093763,0.049337,81.01


In [48]:
# Size summary

size_group = complete_data.groupby(['Size Classification'])

size_students = size_group.sum()["Size"] / size_group.sum()["Cheat"]

size_math_PF = size_group["Pass/Fail Math"].value_counts()
size_passing_math = size_math_PF[1] / size_students
size_reading_PF = size_group["Pass/Fail Reading"].value_counts()
size_passing_reading = size_reading_PF[1] / size_students

In [49]:
# Overall passing rate calculations
size_overall_passing = (size_passing_math + size_passing_reading) / 2

size_summary = pd.DataFrame({"Reading Average": size_group["Reading Score"].mean().round(2),
                                "Math Average": size_group["Reading Score"].mean().round(2),
                                "Percent Passing Math": size_passing_math,
                                "Percent Passing Reading": size_passing_reading,
                                "Overall Passing Rate": size_overall_passing})
size_summary

,Math Average,Overall Passing Rate,Percent Passing Math,Percent Passing Reading,Reading Average
Size Classification,,,,,
Small,83.89,0.121712,0.158489,0.084936,83.89
Mid Sized,82.53,0.058838,0.076617,0.041060,82.53
Large,80.93,0.030076,0.039164,0.020989,80.93


In [50]:
type_group = complete_data.groupby(['Type'])

type_students = type_group.sum()["Size"] / type_group.sum()["Cheat"]

type_math_PF = type_group["Pass/Fail Math"].value_counts()
type_passing_math = type_math_PF[1] / type_students
type_reading_PF = type_group["Pass/Fail Reading"].value_counts()
type_passing_reading = type_reading_PF[1] / type_students

# Overall passing rate calculations
type_overall_passing = (type_passing_math + type_passing_reading) / 2

In [51]:
type_summary = pd.DataFrame({"Reading Average": type_group["Reading Score"].mean().round(2),
                                "Math Average": type_group["Reading Score"].mean().round(2),
                                "Percent Passing Math": type_passing_math,
                                "Percent Passing Reading": type_passing_reading,
                                "Overall Passing Rate": type_overall_passing})
type_summary

,Math Average,Overall Passing Rate,Percent Passing Math,Percent Passing Reading,Reading Average
Type,,,,,
Charter,83.90,0.342679,0.447200,0.238157,83.90
District,80.96,0.144834,0.189011,0.100658,80.96
